# Using Selenium to web-scrape top trails

These trails were scrapped from alltrails.com

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import requests
import re
import glob
import os

In [2]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")

In [3]:
browser = webdriver.Chrome(executable_path='/Users/briandavid/Downloads/chromedriver', options=option)

Loop over each of the the top trails in San Jose

In [136]:
browser.get("https://www.alltrails.com/us/california/san-jose")

In [137]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")

In [138]:
links = [link.get_attribute('href') for link in link_span]

In [4]:
# Acquire Data Method
def get_data(link):
    # Go to page
    browser.get(link)
    completed = browser.find_element_by_class_name('MuiTabs-root.styles-module__slidingTabs___25XGa')
    numComplete = re.findall('(\d+)', completed.text)[3]
    tag_span = browser.find_elements_by_xpath("//span[@class='tag']")
    tags = [x.text for x in tag_span]
    tags = ','.join(list(map(lambda x: x.replace(' ', '-'), tags)))
    detail_data = [data.text for data in browser.find_elements_by_xpath("//span[@class='detail-data xlate-none']")]
    length,elevation = [re.search('([\d.]*)', word.replace(',','')).group(0) for word in detail_data]
    trailname, = [name.text for name in browser.find_elements_by_xpath("//h1[@class='xlate-none styles-module__name___1nEtW']")]
    # Difficulty:
    difficulty =  browser.find_elements_by_xpath("//div[@class='styles-module__info___1Mbn6 styles-module__thin___3YRdx']/span")[0].text
    keys = ["name", "difficulty", "completion", "length", "elevation_gain", "tags"]
    values = [trailname, difficulty, numComplete, float(length), int(elevation), tags]
    data = {k:v for (k,v) in zip(keys,values)}
    return data

In [5]:
def get_num_tags(tags):
    return len(tags.split(','))

In [6]:
from selenium.webdriver.common.action_chains import ActionChains

### Create dataframe from trails - San Jose CA

In [14]:
top_five_sj = list(map(lambda link: get_data(link), links[:5]))

In [7]:
# Column names for the dataframe
col_names = ["name", "difficulty", "completion", "length", "elevation_gain", "tags"]
topfivesj = pd.DataFrame(top_five_sj, columns = col_names, index = list(range(5)))

Count number of tags:

In [16]:
topfivesj['NumberTags'] = topfivesj['tags'].apply(get_num_tags)

Get coordinates:

In [13]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
delay = 1000
icon_src = "https://cdn-assets.alltrails.com/maps/icon-map-marker-start-end@2x.png"
#alt_icon_src = "https://cdn-assets.alltrails.com/assets/icons/maps/icon-map-marker-yellow@2x.png"
alt_icon_src = "https://cdn-assets.alltrails.com/maps/icon-map-marker-start@2x.png"
ending_icon_src = "https://cdn-assets.alltrails.com/maps/icon-map-marker-end@2x.png"

In [12]:
def get_coords(link_list):
    map_coordinates = []
    for link in link_list:
        browser.get(link)
        browser.get(browser.find_elements_by_xpath("//div[@id='map']/div/a")[0].get_attribute('href'))
        time.sleep(5)
        print("done sleeping")
        python_button = browser.find_elements_by_xpath("//div[@class='leaflet-marker-pane']/img")
        for images in python_button:
            if (images.get_attribute('src') == icon_src):
                hoveritem = ActionChains(browser).move_to_element(images)
                hoveritem.perform()
                coords = browser.find_elements_by_xpath("//div[@class='coordinates']")[0].text
                map_coordinates.append(coords)
    return map_coordinates

def get_coords_by_link(link, src):
    browser.get(link)
    browser.get(browser.find_elements_by_xpath("//div[@id='map']/div/a")[0].get_attribute('href'))
    time.sleep(5)
    print("done sleeping")
    python_button = browser.find_elements_by_xpath("//div[@class='leaflet-marker-pane']/img")
    for images in python_button:
        if (images.get_attribute('src') == src):
            hoveritem = ActionChains(browser).move_to_element(images)
            hoveritem.perform()
            coords = browser.find_elements_by_xpath("//div[@class='coordinates']")
            return coords[0].text

In [132]:
map_coordinates

['37.21448, -121.87888',
 '37.39709, -121.79972',
 '37.22608, -121.79894',
 '37.21396, -121.78591',
 '37.20396, -121.86324']

In [134]:
topfivesj['location'] = map_coordinates

Final Dataframe:

In [135]:
topfivesj

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,startingLoc
0,Senador Mine to Mine Hill Loop,moderate,2877,5.3,757,"Dogs-on-leash,Kid-friendly,Hiking,Nature-trips...",13,"37.21448, -121.87888"
1,Alum Rock South Rim Trail,easy,1241,4.3,770,"Kid-friendly,Hiking,Mountain-biking,Nature-tri...",18,"37.39709, -121.79972"
2,Santa Teresa County Park Trail Loop,moderate,700,8.3,1768,"Dogs-on-leash,Hiking,Horseback-riding,Mountain...",9,"37.22608, -121.79894"
3,Coyote Peak and Ridge Trail Loop,moderate,643,3.8,705,"Dogs-on-leash,Kid-friendly,Hiking,Horseback-ri...",13,"37.21396, -121.78591"
4,"Webb Canyon, New Almaden, Cinnabar, Mine Hill,...",moderate,433,5.4,833,"Dogs-on-leash,Hiking,Running,Forest,Wildlife",5,"37.20396, -121.86324"


Creting dataframe from bottom 5 trails

In [139]:
bottom_five_sj = list(map(lambda link:get_data(link),links[63:]))

In [140]:
bottomfivesj = pd.DataFrame(bottom_five_sj, columns = col_names, index = list(range(5)))

In [141]:
bottomfivesj['NumberTags'] = bottomfivesj['tags'].apply(get_num_tags)
bottomfivesj['city'] = "San Jose"
bottomfivesj['rankCat'] = "bottom"

In [146]:
# Get coordinates 
bottomfivesjCoords = get_coords(links[63:])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


Dataframe for bottom five

In [149]:
bottomfivesj

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,startingLoc
0,Communications Hill Trail,easy,59,1.5,137,"Kid-friendly,Walking,Running,Partially-paved,V...",5,San Jose,bottom,"37.28315, -121.85627"
1,"Alum Rock Road, South Rim and Woodland Trail",moderate,152,4.3,761,"Hiking,Nature-trips,Walking,Forest,Views,Wild-...",6,San Jose,bottom,"37.39207, -121.81684"
2,Santa Teresa Mine Loop,easy,157,1.9,200,"Kid-friendly,Hiking,Walking,Running,Partially-...",10,San Jose,bottom,"37.21228, -121.78919"
3,Groesbeck Hills Park Loop,easy,19,0.9,150,"Hiking,Nature-trips,Walking,Views,Wildlife",5,San Jose,bottom,"37.33523, -121.77966"
4,Penitencia Creek Trail,easy,27,5.8,167,"Dog-friendly,Hiking,Road-biking,Running,Lake",5,San Jose,bottom,"37.37673, -121.86115"


In [229]:
topfivesj.to_pickle('topfivesj.pkl')
bottomfivesj.to_pickle('botfivesj.pkl')

### Create dataframe from trails - San Francisco CA

In [111]:
browser.get('https://www.alltrails.com/us/california/san-francisco')

In [112]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
sf_links = [link.get_attribute('href') for link in link_span]

#### Get top 5 trails from SF

In [124]:
top_five_sf = list(map(lambda link: get_data(link), sf_links[:5]))

In [127]:
topfivesf = pd.DataFrame(top_five_sf, columns = col_names, index = list(range(5)))

In [128]:
topfivesf['NumberTags'] = topfivesf['tags'].apply(get_num_tags)
topfivesf['city'] = "SF"
topfivesf['rankCat'] = "top"

In [119]:
topfivesfCoords = get_coords(sf_links[:5])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [203]:
(get_coords_by_link(sf_links[4], ending_icon_src))

done sleeping
https://cdn-assets.alltrails.com/maps/icon-map-marker-end@2x.png
37.80857, -122.4754


In [210]:
topfivesfCoords

['37.78059, -122.51162',
 '37.80744, -122.47459',
 '37.76135, -122.45194',
 '37.80483, -122.47676']

In [121]:
# must use the end-coordinates for the last trail
topfivesfCoords.append(get_coords_by_link(sf_links[4], ending_icon_src))

done sleeping


In [129]:
topfivesf['location'] = topfivesfCoords

Dataframe for top five SF

In [130]:
topfivesf

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Lands End Trail,moderate,5837,3.4,538,"Dogs-on-leash,Kid-friendly,Hiking,Nature-trips...",12,SF,top,"37.78059, -122.51162"
1,Walk Across the Golden Gate Bridge,easy,1611,3.7,193,"Kid-friendly,Nature-trips,Road-biking,Walking,...",11,SF,top,"37.80744, -122.47459"
2,Mount Sutro Loop,moderate,1393,2.0,495,"Dogs-on-leash,Kid-friendly,Mountain-biking,Nat...",10,SF,top,"37.76135, -122.45194"
3,Batteries to Bluffs Trail,easy,1084,2.2,449,"Kid-friendly,Hiking,Nature-trips,Walking,Bird-...",12,SF,top,"37.80483, -122.47676"
4,California Coastal Trail: Lands End to Golden ...,moderate,1855,4.4,659,"Dogs-on-leash,Kid-friendly,Hiking,Walking,Runn...",10,SF,top,"37.80857, -122.4754"


In [131]:
topfivesf.to_pickle('topfivesf.pkl')

#### Get bottom five from SF

In [24]:
bottom_five_sf = list(map(lambda link: get_data(link), sf_links[72:77]))

In [27]:
bottomfivesf = pd.DataFrame(bottom_five_sf, columns = col_names, index = list(range(5)))

In [28]:
bottomfivesf['NumberTags'] = bottomfivesf['tags'].apply(get_num_tags)
bottomfivesf['city'] = "SF"
bottomfivesf['rankCat'] = "bottom"

In [13]:
bottomfivesfCoords = get_coords(sf_links[72:77])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [18]:
# Candlestick Point trail 
bottomfivesfCoords.insert(3, get_coords_by_link(sf_links[75], ending_icon_src))

done sleeping


In [29]:
bottomfivesf['location'] = bottomfivesfCoords

Dataframe for bottm 5 SF

In [30]:
bottomfivesf

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Pine Lake Loop Trail,easy,50,0.9,32,"Hiking,Nature-trips,Walking,Bird-watching,Lake...",6,SF,bottom,"37.73671, -122.48329"
1,"Potrero Hill, Dogpatch and Pier 50",moderate,26,5.1,370,"Nature-trips,Walking,Bird-watching,Partially-p...",6,SF,bottom,"37.75709, -122.40205"
2,Golden Gate Park Polo Field Loop,easy,96,0.8,6,"Dogs-on-leash,Wheelchair-friendly,Kid-friendly...",6,SF,bottom,"37.7668, -122.49484"
3,Candlestick Point State Recreation Area Loop,easy,18,0.7,16,"Dogs-on-leash,Kid-friendly,Hiking,Nature-trips...",9,SF,bottom,"37.71036, -122.38858"
4,Edgehill Mountain Open Space Park Trail,easy,21,0.4,42,"Dogs-on-leash,Kid-friendly,Walking,Running,Par...",5,SF,bottom,"37.74216, -122.45699"


In [65]:
bottomfivesf.to_pickle('botfivesf.pkl')

### Create dataframe from Oakland CA

In [52]:
browser.get('https://www.alltrails.com/us/california/oakland')

In [53]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
oak_links = [link.get_attribute('href') for link in link_span]

#### Top five trails for Oakland

In [39]:
top_five_oak = list(map(lambda link: get_data(link), oak_links[:5]))

In [41]:
topfiveoak = pd.DataFrame(top_five_oak, columns = col_names, index = list(range(5)))

In [42]:
topfiveoak['NumberTags'] = topfiveoak['tags'].apply(get_num_tags)
topfiveoak['city'] = "Oakland"
topfiveoak['rankCat'] = "top"

Get coordinates for trails

In [35]:
topfiveoakCoords = get_coords(oak_links[:5])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [43]:
topfiveoak['location'] = topfiveoakCoords

In [44]:
topfiveoak

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,"French, Bridle and Stream Trail Loop",moderate,2918,9.1,1437,"Dogs-on-leash,Hiking,Nature-trips,Walking,Bird...",11,Oakland,top,"37.83183, -122.18553"
1,"Sunset, Sequoia Bayview, Cindarella, and Sinaw...",moderate,1795,3.2,570,"Dogs-on-leash,Hiking,Mountain-biking,Nature-tr...",10,Oakland,top,"37.81031, -122.18426"
2,"Stream, Mill and French Trail Loop",moderate,1662,4.2,941,"Dog-friendly,Kid-friendly,Hiking,Nature-trips,...",9,Oakland,top,"37.83152, -122.18526"
3,Sequoia Bayview Trail,easy,1535,2.8,278,"Dog-friendly,Kid-friendly,Hiking,Mountain-biki...",12,Oakland,top,"37.80772, -122.17809"
4,Huckleberry Nature Path Trail,easy,986,2.4,439,"Kid-friendly,Hiking,Nature-trips,Bird-watching...",11,Oakland,top,"37.84272, -122.19505"


#### Bottom five Oakland

In [54]:
bottom_five_oak = list(map(lambda link: get_data(link), oak_links[115:120]))

In [56]:
bottomfiveoak = pd.DataFrame(bottom_five_oak, columns = col_names, index = list(range(5)))

In [57]:
bottomfiveoak['NumberTags'] = bottomfiveoak['tags'].apply(get_num_tags)
bottomfiveoak['city'] = "Oakland"
bottomfiveoak['rankCat'] = "bottom"

In [47]:
bottomfiveoakCoords = get_coords(oak_links[115:120])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [60]:
# Use single link for Dunn,West Ridge Trail
bottomfiveoakCoords.insert(1, get_coords_by_link(oak_links[116], icon_src))

done sleeping


In [61]:
bottomfiveoakCoords

['37.81015, -122.19879',
 '37.80303, -122.17765',
 '37.73913, -122.09782',
 '37.74074, -122.09412',
 '37.84031, -122.18105']

In [62]:
bottomfiveoak['location'] = bottomfiveoakCoords

In [63]:
bottomfiveoak

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Anthony Chabot Regional Park Cycle Loop,moderate,11,28.3,2936,"Road-biking,Forest,Lake,Paved,Views,Wild-flowe...",7,Oakland,bottom,"37.81015, -122.19879"
1,"Dunn, West Ridge, Redwood Peak, French, West R...",moderate,153,10.8,2073,"Hiking,Nature-trips,Walking,Running,Forest,Wil...",7,Oakland,bottom,"37.80303, -122.17765"
2,Lost Ridge to Quail Trail,easy,9,2.8,449,"Dog-friendly,Hiking,Nature-trips,Lake,Views,Wi...",6,Oakland,bottom,"37.73913, -122.09782"
3,"Towhee, Columbine, and Quail Trail Loop",easy,17,4.7,718,"Hiking,Running,Forest,Views,Wildlife",5,Oakland,bottom,"37.74074, -122.09412"
4,McCosker Loop [CLOSED],moderate,80,2.5,538,"Dogs-on-leash,Kid-friendly,Hiking,Nature-trips...",12,Oakland,bottom,"37.84031, -122.18105"


In [64]:
topfiveoak.to_pickle('topfiveoak.pkl')
bottomfiveoak.to_pickle('botfiveoak.pkl')

### Create dataframe from Mill Valley CA

In [85]:
browser.get('https://www.alltrails.com/us/california/mill-valley')

In [86]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
millvalley_links = [link.get_attribute('href') for link in link_span]

#### Top five Mill Valley

In [76]:
top_five_mill = list(map(lambda link: get_data(link), millvalley_links[:5]))

In [81]:
topfivemill = pd.DataFrame(top_five_mill, columns = col_names, index = list(range(5)))

In [82]:
topfivemill['NumberTags'] = topfivemill['tags'].apply(get_num_tags)
topfivemill['city'] = "Mill Valley"
topfivemill['rankCat'] = "top"

Get coords for top five 

In [69]:
top5millCoords = get_coords(millvalley_links[:3])

done sleeping
done sleeping
done sleeping


In [73]:
# idiv
# zoomed prior
icon_to_hover = browser.find_elements_by_xpath("//*[@id='map']/div[1]/div[2]/div[3]/img[19]")
hoveritem = ActionChains(browser).move_to_element(icon_to_hover[0])
hoveritem.perform()
top5millCoords.append(browser.find_elements_by_xpath("//div[@class='coordinates']")[0].text)
top5millCoords.append(get_coords_by_link(millvalley_links[4], icon_src))

done sleeping


In [83]:
topfivemill['location'] = top5millCoords

Dataframe for top five Mill Valley

In [84]:
topfivemill

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Muir Woods Trail,easy,4208,1.3,42,"Wheelchair-friendly,Kid-friendly,Stroller-frie...",14,Mill Valley,top,"37.89134, -122.56938"
1,Dipsea Trail,hard,1650,9.7,2545,"Hiking,Nature-trips,Bird-watching,Running,Beac...",11,Mill Valley,top,"37.89153, -122.56926"
2,Cataract Falls and Potrero Meadows Loop (Catar...,moderate,1130,8.4,1837,"Dogs-on-leash,Hiking,Nature-trips,Walking,Bird...",14,Mill Valley,top,"37.91085, -122.61282"
3,Dipsea Trail to Steep Ravine Trail Loop from P...,moderate,1374,3.6,948,"Kid-friendly,Hiking,Nature-trips,Walking,Bird-...",11,Mill Valley,top,"37.90346, -122.60402"
4,Pirate's Cove,moderate,1306,3.6,820,"Dogs-on-leash,Hiking,Nature-trips,Walking,Bird...",11,Mill Valley,top,"37.8614, -122.57545"


In [88]:
bottom_five_mill = list(map(lambda link: get_data(link), millvalley_links[105:]))

In [89]:
bottomfivemill = pd.DataFrame(bottom_five_mill, columns = col_names, index = list(range(5)))

In [90]:
bottomfivemill['NumberTags'] = bottomfivemill['tags'].apply(get_num_tags)
bottomfivemill['city'] = "Mill Valley"
bottomfivemill['rankCat'] = "bottom"

In [87]:
bot5millCoords = get_coords(millvalley_links[105:])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [91]:
bottomfivemill['location'] = bot5millCoords

In [92]:
bottomfivemill

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Pantoll to Mount Theater to O'Rourke's to Matt...,moderate,41,3.4,643,"Hiking,Nature-trips,Bird-watching,Running,Fore...",7,Mill Valley,bottom,"37.90381, -122.60434"
1,Muir Woods Cycle Loop,easy,9,12.8,1496,"Road-biking,Forest,Paved,Views,Wild-flowers,Wi...",6,Mill Valley,bottom,"37.90642, -122.54795"
2,Mill Valley to East Peak via Dipsea Steps,hard,56,10.2,2847,"Hiking,Nature-trips,Forest,Views,Wild-flowers",5,Mill Valley,bottom,"37.90566, -122.5538"
3,Stinson Beach Cycle Loop,moderate,4,23.0,3051,"Road-biking,Beach,Forest,Paved,Views,Wild-flow...",7,Mill Valley,bottom,"37.90653, -122.548"
4,"Gravity Car Road, Northside, Benstein and Matt...",hard,17,10.6,1666,"Hiking,Running,Forest,Views,Wildlife",5,Mill Valley,bottom,"37.911, -122.57734"


In [93]:
topfivemill.to_pickle('topfivemill.pkl')
bottomfivemill.to_pickle('botfivemill.pkl')

### Create dataframe from Redwood City CA

In [102]:
browser.get('https://www.alltrails.com/us/california/redwood-city')

In [103]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
rwcity_links = [link.get_attribute('href') for link in link_span]

top five in Redwood City

In [98]:
top_five_rwc = list(map(lambda link: get_data(link), rwcity_links[:5]))
topfiverwc = pd.DataFrame(top_five_rwc, columns = col_names, index = list(range(5)))

In [99]:
topfiverwc['NumberTags'] = topfiverwc['tags'].apply(get_num_tags)
topfiverwc['city'] = "Redwood City"
topfiverwc['rankCat'] = "top"

In [96]:
top5rwcCoords = get_coords(rwcity_links[:5])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [100]:
topfiverwc['location'] = top5rwcCoords

In [101]:
topfiverwc

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Whittemore Gulch and Purisima Creek Loop Trail,hard,1968,9.3,2109,"Hiking,Horseback-riding,Nature-trips,Bird-watc...",13,Redwood City,top,"37.45006, -122.33866"
1,Whittemore Gulch and Harkins Ridge Loop Trail,hard,1486,7.6,1853,"Hiking,Mountain-biking,Nature-trips,Running,Fo...",10,Redwood City,top,"37.44992, -122.33874"
2,Crystal Springs and Dean Trail,moderate,1732,5.0,935,"Kid-friendly,Hiking,Nature-trips,Walking,Runni...",11,Redwood City,top,"37.44169, -122.29066"
3,Purisima Creek Redwoods North Ridge Trail,moderate,1035,4.4,1361,"Hiking,Horseback-riding,Mountain-biking,Nature...",17,Redwood City,top,"37.44976, -122.33891"
4,Edgewood Trail Loop,moderate,1133,3.1,590,"Kid-friendly,Hiking,Nature-trips,Walking,Bird-...",10,Redwood City,top,"37.4731, -122.27817"


In [105]:
bot5rwcCoords = get_coords(rwcity_links[79:])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [107]:
bot_five_rwc = list(map(lambda link: get_data(link), rwcity_links[79:]))

In [108]:
botfiverwc = pd.DataFrame(bot_five_rwc, columns = col_names, index = list(range(5)))

In [12]:
bot_five_rwc['NumberTags'] = bot_five_rwc['tags'].apply(get_num_tags)
bot_five_rwc['city'] = "Redwood City"
bot_five_rwc['rankCat'] = "bottom"

In [ ]:
botfiverwc[location] = bot5rwcCoords

In [11]:
botfiverwc

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Old La Honda Road,hard,19,19.9,2401,"Nature-trips,Road-biking,Views,No-dogs",4,Redwood City,bottom,"37.40736, -122.25689"
1,Timerberview and Gordon Mill Trail,moderate,105,5.8,1443,"Hiking,Nature-trips,Forest,Views,Wildlife",5,Redwood City,bottom,"37.40598, -122.30461"
2,Crystal Springs Trail,moderate,31,3.9,439,"Hiking,Nature-trips,Walking,Running",4,Redwood City,bottom,"37.46065, -122.2938"
3,Bair Island Reserve & Corkscrew Slough,easy,26,1.7,13,"Kid-friendly,Paddle-sports,Walking,No-dogs",4,Redwood City,bottom,"37.50198, -122.21455"
4,Summit Springs Loop,easy,7,1.7,682,"Kid-friendly,Hiking,Horseback-riding,Nature-tr...",13,Redwood City,bottom,"37.42364, -122.27602"


### Top trails near Fremont

In [27]:
browser.get('https://www.alltrails.com/us/california/fremont')

In [32]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
fremont_links = [link.get_attribute('href') for link in link_span]

In [17]:
top5fremontCoords = get_coords(fremont_links[:5])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [18]:
top5fremontCoords

['37.50438, -121.90833',
 '37.52716, -121.91312',
 '37.50432, -121.9085',
 '37.55359, -122.09011',
 '37.55393, -122.09124']

In [25]:
bot5fremontCoords = get_coords(fremont_links[31:])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [26]:
bot5fremontCoords

['37.55406, -122.0908',
 '37.53263, -122.07325',
 '37.55387, -122.09079',
 '37.55407, -122.0908',
 '37.5539, -122.09095']

In [30]:
top_five_fremont = list(map(lambda link: get_data(link), fremont_links[:5]))

In [33]:
bot_five_fremont = list(map(lambda link: get_data(link), fremont_links[31:]))

In [34]:
topfivefremont = pd.DataFrame(top_five_fremont, columns = col_names, index = list(range(5)))
botfivefremont = pd.DataFrame(bot_five_fremont, columns = col_names, index = list(range(5)))

topfivefremont['NumberTags'] = topfivefremont['tags'].apply(get_num_tags)
topfivefremont['city'] = "Fremont"
topfivefremont['rankCat'] = "top"
topfivefremont['location'] = top5fremontCoords

botfivefremont['NumberTags'] = botfivefremont['tags'].apply(get_num_tags)
botfivefremont['city'] = "Fremont"
botfivefremont['rankCat'] = "bottom"
botfivefremont['location'] = bot5fremontCoords

In [35]:
topfivefremont

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Mission Peak Loop from Stanford Avenue Staging...,hard,6487,5.8,2135,"Dog-friendly,Hiking,Horseback-riding,Mountain-...",11,Fremont,top,"37.50438, -121.90833"
1,Mission Peak from Ohlone College,hard,3048,7.3,2145,"Dog-friendly,Backpacking,Camping,Hiking,Runnin...",11,Fremont,top,"37.52716, -121.91312"
2,Hidden Valley Trail,hard,2686,6.1,1847,"Dogs-on-leash,Hiking,Nature-trips,Running,View...",9,Fremont,top,"37.50432, -121.9085"
3,Bayview and Meadowlark Trail Loop,easy,952,4.9,337,"Dog-friendly,Wheelchair-friendly,Kid-friendly,...",17,Fremont,top,"37.55359, -122.09011"
4,Quail and Bayview Loop,easy,653,3.1,170,"Hiking,Nature-trips,Walking,Bird-watching,Fee",5,Fremont,top,"37.55393, -122.09124"


In [36]:
botfivefremont

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,"Chochenyo, Red Hill and Meadowlark Loop",moderate,90,8.5,672,"Dogs-on-leash,Kid-friendly,Backpacking,Hiking,...",17,Fremont,bottom,"37.55406, -122.0908"
1,Apay Way to No Name to Red Hills Loop,moderate,57,9.8,603,"Dogs-on-leash,Kid-friendly,Hiking,Mountain-bik...",10,Fremont,bottom,"37.53263, -122.07325"
2,"Nike, Bayview, Pelican, No Name, Soaproot Loop",easy,77,7.7,259,"Hiking,Nature-trips,Walking,Bird-watching,Views",5,Fremont,bottom,"37.55387, -122.09079"
3,Lizard Rock Trail,easy,137,1.0,36,"Kid-friendly,Hiking,Nature-trips,Walking,Bird-...",10,Fremont,bottom,"37.55407, -122.0908"
4,"Quail Trail, Glider Hill, Soaproot Loop",easy,44,1.1,200,"Hiking,Nature-trips,Walking,Bird-watching,View...",7,Fremont,bottom,"37.5539, -122.09095"


In [37]:
topfivefremont.to_pickle("topfivefremont.pkl")
botfivefremont.to_pickle("botfivefremont.pkl")

In [44]:
filenames = (glob.glob('[botp]*.pkl'))

In [45]:
topbot = ['top', 'bot']

In [46]:
filenames = (sorted(filenames, key = (lambda x: (re.search('(\w{3})five(\w+).pkl',x).group(2), topbot.index(re.search('(\w{3})five(\w+).pkl',x).group(1))))))

In [47]:
df = (pd.concat((pd.read_pickle(file) for file in filenames)))

In [160]:
x = [pd.read_pickle(file) for file in filenames]

In [48]:
df

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Seaview and Big Springs Trails Loop,moderate,2447,4.3,984,"Dog-friendly,Kid-friendly,Hiking,Horseback-rid...",12,Berkeley,top,"37.9006, -122.24968"
1,"Panoramic Hill, Chaparral Peak and Strawberry ...",hard,1558,6.2,1515,"Dog-friendly,Hiking,Nature-trips,Bird-watching...",10,Berkeley,top,"37.8627, -122.24408"
2,"Wildcat Gorge, Meadows Canyon and Curran Trail",moderate,1583,3.3,498,"Dog-friendly,Kid-friendly,Hiking,Horseback-rid...",12,Berkeley,top,"37.89572, -122.25213"
3,Strawberry Canyon Fire Trail,moderate,1161,7.4,777,"Dog-friendly,Kid-friendly,Hiking,Walking,Runni...",7,Berkeley,top,"37.87251, -122.24178"
4,Upper and Lower Big Springs and Seaview Trails,moderate,1397,2.9,652,"Dogs-on-leash,Kid-friendly,Hiking,Mountain-bik...",5,Berkeley,top,"37.89035, -122.23651"
...,...,...,...,...,...,...,...,...,...,...
0,Mount Diablo via Wall Point Road and Summit Tr...,hard,82,13.8,3490,"Hiking,Views,Closed,No-dogs",4,Walnut Creek,bottom,"37.86249, -121.97916"
1,Diablo Foothills Crests and Creeks [CLOSED],moderate,22,13.0,2440,"Hiking,Nature-trips,Bird-watching,Views,Wild-f...",7,Walnut Creek,bottom,"37.89394, -121.99418"
2,Diablo Foothills to Summit Loop [CLOSED],hard,30,16.0,3887,"Hiking,Nature-trips,Bird-watching,Views,Wild-f...",7,Walnut Creek,bottom,"37.89406, -121.99448"
3,Macedo Ranch Loop [CLOSED],hard,54,8.4,1801,"Hiking,Nature-trips,Bird-watching,Running,View...",9,Walnut Creek,bottom,"37.86249, -121.97916"


In [49]:
df.to_excel("hikingtrails.xlsx")

### Top trails near Los Altos 

In [22]:
browser.get("https://www.alltrails.com/us/california/los-altos")

In [24]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
losaltos_links = [link.get_attribute('href') for link in link_span]

In [15]:
top5losaltosCoords = get_coords(losaltos_links[:5])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [16]:
top5losaltosCoords

['37.35211, -122.13802',
 '37.3522, -122.1382',
 '37.31203, -122.1768',
 '37.32546, -122.17868',
 '37.35215, -122.13801']

In [18]:
bot5losaltosCoords = get_coords(losaltos_links[24:29])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [19]:
bot5losaltosCoords

['37.32602, -122.17999',
 '37.35225, -122.1598',
 '37.35173, -122.16028',
 '37.34999, -122.16082']

In [20]:
bot5losaltosCoords.insert(1,get_coords_by_link(losaltos_links[25], alt_icon_src))

done sleeping


In [21]:
bot5losaltosCoords

['37.32602, -122.17999',
 '37.29051, -122.15424',
 '37.35225, -122.1598',
 '37.35173, -122.16028',
 '37.34999, -122.16082']

In [25]:
top_five_losaltos = list(map(lambda link: get_data(link), losaltos_links[:5]))

In [26]:
bot_five_losaltos = list(map(lambda link: get_data(link), losaltos_links[24:29]))

In [28]:
topfivelostaltos = pd.DataFrame(top_five_losaltos, columns = col_names, index = list(range(5)))
botfivelostaltos = pd.DataFrame(bot_five_losaltos, columns = col_names, index = list(range(5)))

topfivelostaltos['NumberTags'] = topfivelostaltos['tags'].apply(get_num_tags)
topfivelostaltos['city'] = "Los Altos"
topfivelostaltos['rankCat'] = "top"
topfivelostaltos['location'] = top5losaltosCoords

botfivelostaltos['NumberTags'] = botfivelostaltos['tags'].apply(get_num_tags)
botfivelostaltos['city'] = "Los Altos"
botfivelostaltos['rankCat'] = "bottom"
botfivelostaltos['location'] = bot5losaltosCoords

In [29]:
topfivelostaltos

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Black Mountain via Rhus Ridge,moderate,608,9.2,2506,"Hiking,Walking,Running,Forest,Views,Wild-flowe...",8,Los Altos,top,"37.35211, -122.13802"
1,Rhus Ridge and Creek Trail Loop,moderate,363,7.1,1669,"Hiking,Nature-trips,Running,Forest,Views,Wild-...",8,Los Altos,top,"37.3522, -122.1382"
2,Horseshoe Lake - Alpine Pond - Borel Hill,moderate,220,6.1,1079,"Kid-friendly,Hiking,Horseback-riding,Mountain-...",14,Los Altos,top,"37.31203, -122.1768"
3,Monte Bello Black Mountain Loop,moderate,414,5.4,1007,"Kid-friendly,Hiking,Nature-trips,Walking,Fores...",10,Los Altos,top,"37.32546, -122.17868"
4,Rhus Ridge Trail,moderate,681,1.7,626,"Hiking,Walking,Views,Wild-flowers,No-dogs",5,Los Altos,top,"37.35215, -122.13801"


In [30]:
botfivelostaltos

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Los Trancos Loop Trail,easy,71,2.7,452,"Kid-friendly,Hiking,Nature-trips,Walking,Bird-...",10,Los Altos,bottom,"37.32602, -122.17999"
1,Long Ridge to Peters Creek,hard,13,18.3,4064,"Hiking,Nature-trips,Views,Wild-flowers",4,Los Altos,bottom,"37.29051, -122.15424"
2,Long Bunny Loop Trail,moderate,21,2.4,488,"Hiking,Nature-trips,Bird-watching,Running,Fore...",7,Los Altos,bottom,"37.35225, -122.1598"
3,"Hostel Trail, Pipeline Trail and Creek Trail Loop",easy,44,2.1,449,"Hiking,Nature-trips,Walking,Bird-watching,Fore...",7,Los Altos,bottom,"37.35173, -122.16028"
4,Hostel Trail to Toyon Trail,easy,31,1.1,400,"Kid-friendly,Hiking,Nature-trips,Walking,Bird-...",9,Los Altos,bottom,"37.34999, -122.16082"


In [31]:
topfivelostaltos.to_pickle("topfivelostaltos.pkl")
botfivelostaltos.to_pickle("botfivelostaltos.pkl")

### Berkeley Trails

In [46]:
browser.get("https://www.alltrails.com/us/california/berkeley")

In [48]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
berk_links = [link.get_attribute('href') for link in link_span]

In [37]:
#top5berkCoords = get_coords(berk_links[:5])
top5berkCoords = []
browser.get(berk_links[0])
top5berkCoords.insert(0,get_coords_by_link(berk_links[0], icon_src))
#top5berkCoords.insert(1,get_coords_by_link(berk_links[1], icon_src))

done sleeping


In [38]:
browser.get(berk_links[1])

In [39]:
icon_to_hover = browser.find_elements_by_xpath("//*[@id='map']/div[1]/div[2]/div[3]/img[13]")
hoveritem = ActionChains(browser).move_to_element(icon_to_hover[0])
hoveritem.perform()
top5berkCoords.append(browser.find_elements_by_xpath("//div[@class='coordinates']")[0].text)

In [42]:
reminaingtop5 = get_coords(berk_links[2:5])

done sleeping
done sleeping
done sleeping


In [44]:
top5berkCoords = [*top5berkCoords, *reminaingtop5]

In [45]:
top5berkCoords

['37.9006, -122.24968',
 '37.8627, -122.24408',
 '37.89572, -122.25213',
 '37.87251, -122.24178',
 '37.89035, -122.23651']

In [50]:
bot5berkCoords = get_coords(berk_links[60:])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [51]:
bot5berkCoords

['37.87515, -122.26017',
 '37.8751, -122.26007',
 '37.88182, -122.27111',
 '37.87352, -122.25945']

In [52]:
bot5berkCoords.insert(0,get_coords_by_link(berk_links[60], alt_icon_src))

done sleeping


In [54]:
top_five_berk = list(map(lambda link: get_data(link), berk_links[:5]))

In [55]:
bot_five_berk = list(map(lambda link: get_data(link), berk_links[60:]))

In [56]:
topfiveberk = pd.DataFrame(top_five_berk, columns = col_names, index = list(range(5)))
botfiveberk = pd.DataFrame(bot_five_berk, columns = col_names, index = list(range(5)))

topfiveberk['NumberTags'] = topfiveberk['tags'].apply(get_num_tags)
topfiveberk['city'] = "Berkeley"
topfiveberk['rankCat'] = "top"
topfiveberk['location'] = top5berkCoords

botfiveberk['NumberTags'] = botfiveberk['tags'].apply(get_num_tags)
botfiveberk['city'] = "Berkeley"
botfiveberk['rankCat'] = "bottom"
botfiveberk['location'] = bot5berkCoords

In [57]:
topfiveberk

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Seaview and Big Springs Trails Loop,moderate,2447,4.3,984,"Dog-friendly,Kid-friendly,Hiking,Horseback-rid...",12,Berkeley,top,"37.9006, -122.24968"
1,"Panoramic Hill, Chaparral Peak and Strawberry ...",hard,1558,6.2,1515,"Dog-friendly,Hiking,Nature-trips,Bird-watching...",10,Berkeley,top,"37.8627, -122.24408"
2,"Wildcat Gorge, Meadows Canyon and Curran Trail",moderate,1583,3.3,498,"Dog-friendly,Kid-friendly,Hiking,Horseback-rid...",12,Berkeley,top,"37.89572, -122.25213"
3,Strawberry Canyon Fire Trail,moderate,1161,7.4,777,"Dog-friendly,Kid-friendly,Hiking,Walking,Runni...",7,Berkeley,top,"37.87251, -122.24178"
4,Upper and Lower Big Springs and Seaview Trails,moderate,1397,2.9,652,"Dogs-on-leash,Kid-friendly,Hiking,Mountain-bik...",5,Berkeley,top,"37.89035, -122.23651"


In [58]:
botfiveberk

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Pine Tree Trail and Laurel Canyon Loop,moderate,38,1.5,288,"Kid-friendly,Hiking,Nature-trips,Running,Fores...",9,Berkeley,bottom,"37.90899, -122.26517"
1,"Wildcat Canyon, El Toyonal, Lomas Contadas, an...",moderate,10,16.6,1916,"Nature-trips,Road-biking,Forest,Paved,Views,Wi...",7,Berkeley,bottom,"37.87515, -122.26017"
2,Wildcat Canyon and South Park Cycling Route,moderate,15,10.3,1515,"Nature-trips,Road-biking,Forest,Paved,Views,Wi...",7,Berkeley,bottom,"37.8751, -122.26007"
3,East Bay Hills Cycle Route,hard,4,39.6,4517,"Mountain-biking,Forest,Lake,Partially-paved,Views",5,Berkeley,bottom,"37.88182, -122.27111"
4,"Observatory Hill, Lake Chabot and Claremont Ca...",moderate,1,48.5,4829,"Nature-trips,Road-biking,Paved,Views",4,Berkeley,bottom,"37.87352, -122.25945"


In [59]:
topfiveberk.to_pickle("topfiveberk.pkl")
botfiveberk.to_pickle("botfiveberk.pkl")

### Trails for Pacifica

In [64]:
browser.get("https://www.alltrails.com/us/california/pacifica")

In [61]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
pac_links = [link.get_attribute('href') for link in link_span]

In [62]:
pactop5Coords = get_coords(pac_links[:5])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [67]:
pacbot5Coords = get_coords(pac_links[22:27])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [71]:
pacbot5Coords

['37.57891, -122.4746',
 '37.57891, -122.47459',
 '37.59328, -122.50601',
 '37.59699, -122.50387']

In [76]:
pacbot5Coords.insert(3,get_coords_by_link(pac_links[25], icon_src))

done sleeping


In [80]:
browser.get(pac_links[1])

In [81]:
pactop5Coords.insert(0,get_coords_by_link(pac_links[0], icon_src))

done sleeping


In [82]:
pactop5Coords

['37.61413, -122.48807',
 '37.57837, -122.47549',
 '37.56294, -122.51261',
 '37.58492, -122.50778',
 '37.61735, -122.48322']

In [83]:
top_five_pac = list(map(lambda link: get_data(link), pac_links[:5]))

In [85]:
bot_five_pac = list(map(lambda link: get_data(link), pac_links[22:27]))

In [87]:
topfivepac = pd.DataFrame(top_five_pac, columns = col_names, index = list(range(5)))
botfivepac = pd.DataFrame(bot_five_pac, columns = col_names, index = list(range(5)))

topfivepac['NumberTags'] = topfivepac['tags'].apply(get_num_tags)
topfivepac['city'] = "Pacifica"
topfivepac['rankCat'] = "top"
topfivepac['location'] = pactop5Coords

botfivepac['NumberTags'] = botfivepac['tags'].apply(get_num_tags)
botfivepac['city'] = "Pacifica"
botfivepac['rankCat'] = "bottom"
botfivepac['location'] = pacbot5Coords

In [88]:
topfivepac

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Mori Point Loop Trail,moderate,1656,2.5,396,"Dogs-on-leash,Kid-friendly,Hiking,Nature-trips...",13,Pacifica,top,"37.61413, -122.48807"
1,Montara Mountain Trail,moderate,1437,6.4,1725,"Hiking,Horseback-riding,Nature-trips,Bird-watc...",10,Pacifica,top,"37.57837, -122.47549"
2,Gray Whale Cove Trail,easy,986,2.2,269,"Dog-friendly,Kid-friendly,Hiking,Nature-trips,...",10,Pacifica,top,"37.56294, -122.51261"
3,Pedro Point Headlands Trail,moderate,613,2.4,610,"Dog-friendly,Hiking,Nature-trips,Walking,Bird-...",9,Pacifica,top,"37.58492, -122.50778"
4,Sweeney Ridge Trail via Mori Ridge,moderate,1091,4.6,1085,"Dogs-on-leash,Hiking,Mountain-biking,Nature-tr...",11,Pacifica,top,"37.61735, -122.48322"


In [89]:
botfivepac

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location
0,Valley View Trail Loop,moderate,172,1.9,433,"Kid-friendly,Hiking,Nature-trips,Walking,Bird-...",9,Pacifica,bottom,"37.57891, -122.4746"
1,Weiler Ranch Road Trail,easy,319,1.9,134,"Kid-friendly,Hiking,Mountain-biking,Walking,Ru...",10,Pacifica,bottom,"37.57891, -122.47459"
2,Pedro Point Headlands Trail,easy,60,1.4,521,"Kid-friendly,Hiking,Walking,Running,Views,Wild...",6,Pacifica,bottom,"37.59328, -122.50601"
3,The Mile to San Pedro Mountain Trail to Burnside,moderate,80,3.7,1276,"Dogs-on-leash,Hiking,Mountain-biking,Running,V...",6,Pacifica,bottom,"37.58303, -122.4975"
4,San Pedro Beach to Devil's Slide,moderate,26,6.7,1115,"Hiking,Walking,Bird-watching,Running,Beach,Vie...",8,Pacifica,bottom,"37.59699, -122.50387"


### Trails for Walnut Creek

In [8]:
browser.get("https://www.alltrails.com/us/california/walnut-creek")

In [10]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
wal_links = [link.get_attribute('href') for link in link_span]

In [14]:
waltop5Coords = get_coords(wal_links[:5])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [18]:
walbot5Coords = get_coords(wal_links[38:])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [20]:
top_five_wal = list(map(lambda link: get_data(link), wal_links[:5]))

In [22]:
bot_five_wal = list(map(lambda link: get_data(link), wal_links[38:]))

In [23]:
topfivewal = pd.DataFrame(top_five_wal, columns = col_names, index = list(range(5)))
botfivewal = pd.DataFrame(bot_five_wal, columns = col_names, index = list(range(5)))

topfivewal['NumberTags'] = topfivewal['tags'].apply(get_num_tags)
topfivewal['city'] = "Walnut Creek"
topfivewal['rankCat'] = "top"
topfivewal['location'] = waltop5Coords

botfivewal['NumberTags'] = botfivewal['tags'].apply(get_num_tags)
botfivewal['city'] = "Walnut Creek"
botfivewal['rankCat'] = "bottom"
botfivewal['location'] = walbot5Coords

In [24]:
topfivewal.to_pickle('topfivewal.pkl')
botfivewal.to_pickle('botfivewal.pkl')

### Trails for Santa Rosa

In [25]:
browser.get("https://www.alltrails.com/us/california/santa-rosa")

In [26]:
link_span = browser.find_elements_by_xpath("//div[@class='styles-module__containerDescriptive___3aZqQ styles-module__trailCard___2oHiP']/a")
sr_links = [link.get_attribute('href') for link in link_span]

In [27]:
srtop5Coords = get_coords(sr_links[:5])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [33]:
srbot5Coords = get_coords(sr_links[44:])

done sleeping
done sleeping
done sleeping
done sleeping
done sleeping


In [36]:
srbot5Coords.insert(4, get_coords_by_link(sr_links[48], alt_icon_src))

done sleeping


In [37]:
top_five_sr = list(map(lambda link: get_data(link), sr_links[:5]))

In [38]:
bot_five_sr = list(map(lambda link: get_data(link), sr_links[44:]))

In [39]:
topfivesr = pd.DataFrame(top_five_sr, columns = col_names, index = list(range(5)))
botfivesr = pd.DataFrame(bot_five_sr, columns = col_names, index = list(range(5)))

topfivesr['NumberTags'] = topfivesr['tags'].apply(get_num_tags)
topfivesr['city'] = "Santa Rosa"
topfivesr['rankCat'] = "top"
topfivesr['location'] = srtop5Coords

botfivesr['NumberTags'] = botfivesr['tags'].apply(get_num_tags)
botfivesr['city'] = "Santa Rosa"
botfivesr['rankCat'] = "bottom"
botfivesr['location'] = srbot5Coords

In [42]:
topfivesr.to_pickle('topfivesr.pkl')
botfivesr.to_pickle('botfivesr.pkl')

In [69]:
test = df.head().copy()

In [77]:
dogs, paved, kf, hk, hr, *other = suitability_tags

In [61]:
def contain_tag(trail, tag):
    return tag in trail

In [78]:
test['dogs'] = test['tags'].apply(contain_tag, tag = dogs)
test['paved'] = test['tags'].apply(contain_tag, tag = paved)
test['hr'] = test['tags'].apply(contain_tag, tag = hr)

In [88]:
for tag_name in suitability_tags:
    df[tag_name] = df['tags'].apply(contain_tag, tag = tag_name)

In [90]:
for tag_name in attraction_tags:
    df[tag_name] = df['tags'].apply(contain_tag, tag = tag_name)

In [92]:
for tag_name in activity_tags:
    df[tag_name] = df['tags'].apply(contain_tag, tag = tag_name)

In [93]:
df

,name,difficulty,completion,length,elevation_gain,tags,NumberTags,city,rankCat,location,...,Paddle-sports,Road-biking,Rock-climbing,Scenic-driving,Snowshoeing,Skiing,Running,X-Country-skiing,Walking,Via-ferrata
0,Seaview and Big Springs Trails Loop,moderate,2447,4.3,984,"Dog-friendly,Kid-friendly,Hiking,Horseback-rid...",12,Berkeley,top,"37.9006, -122.24968",...,False,False,False,False,False,False,True,False,True,False
1,"Panoramic Hill, Chaparral Peak and Strawberry ...",hard,1558,6.2,1515,"Dog-friendly,Hiking,Nature-trips,Bird-watching...",10,Berkeley,top,"37.8627, -122.24408",...,False,False,False,False,False,False,True,False,False,False
2,"Wildcat Gorge, Meadows Canyon and Curran Trail",moderate,1583,3.3,498,"Dog-friendly,Kid-friendly,Hiking,Horseback-rid...",12,Berkeley,top,"37.89572, -122.25213",...,False,False,False,False,False,False,True,False,False,False
3,Strawberry Canyon Fire Trail,moderate,1161,7.4,777,"Dog-friendly,Kid-friendly,Hiking,Walking,Runni...",7,Berkeley,top,"37.87251, -122.24178",...,False,False,False,False,False,False,True,False,True,False
4,Upper and Lower Big Springs and Seaview Trails,moderate,1397,2.9,652,"Dogs-on-leash,Kid-friendly,Hiking,Mountain-bik...",5,Berkeley,top,"37.89035, -122.23651",...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Mount Diablo via Wall Point Road and Summit Tr...,hard,82,13.8,3490,"Hiking,Views,Closed,No-dogs",4,Walnut Creek,bottom,"37.86249, -121.97916",...,False,False,False,False,False,False,False,False,False,False
1,Diablo Foothills Crests and Creeks [CLOSED],moderate,22,13.0,2440,"Hiking,Nature-trips,Bird-watching,Views,Wild-f...",7,Walnut Creek,bottom,"37.89394, -121.99418",...,False,False,False,False,False,False,False,False,False,False
2,Diablo Foothills to Summit Loop [CLOSED],hard,30,16.0,3887,"Hiking,Nature-trips,Bird-watching,Views,Wild-f...",7,Walnut Creek,bottom,"37.89406, -121.99448",...,False,False,False,False,False,False,False,False,False,False
3,Macedo Ranch Loop [CLOSED],hard,54,8.4,1801,"Hiking,Nature-trips,Bird-watching,Running,View...",9,Walnut Creek,bottom,"37.86249, -121.97916",...,False,False,False,False,False,False,True,False,False,False


## Scraping tags from alltrails

Tags are characteristics that can be applied to a hiking trail

In [4]:
browser.get("https://www.alltrails.com/explore/us/california/san-jose")

In [5]:
# get all activity-related tags
activity_div = browser.find_elements_by_xpath("//*[@id='spring-popper']/div/div/div[2]/div[2]/span/div/div")
activity_div_children = activity_div[0].find_elements_by_xpath(".//*")
activity_tags = list(map(lambda tag: tag.text.replace(' ', '-'), activity_div_children))

In [6]:
# get all attraction-related tags
attraction_div = browser.find_elements_by_xpath("//*[@id='spring-popper']/div/div/div[3]/div[2]/span/div/div")
attraction_div_children = attraction_div[0].find_elements_by_xpath(".//*")
attraction_tags = list(map(lambda tag: tag.text.replace(' ', '-'), attraction_div_children))

In [7]:
# get all suitability-related tags
suitability_div = browser.find_elements_by_xpath("//*[@id='spring-popper']/div/div/div[4]/div[2]/span/div/div")
suitability_div_children = suitability_div[0].find_elements_by_xpath(".//*")
suitability_tags = list(map(lambda tag: tag.text.replace(' ', '-'), suitability_div_children))

In [8]:
activity_tags

['Backpacking',
 'Bike-touring',
 'Bird-watching',
 'Camping',
 'Fishing',
 'Hiking',
 'Horseback-riding',
 'Mountain-biking',
 'Nature-trips',
 'OHV/Off-road-driving',
 'Paddle-sports',
 'Road-biking',
 'Rock-climbing',
 'Scenic-driving',
 'Snowshoeing',
 'Skiing',
 'Running',
 'X-Country-skiing',
 'Walking',
 'Via-ferrata']

In [9]:
attraction_tags

['Beach',
 'Cave',
 'City-walk',
 'Event',
 'Forest',
 'Historic-site',
 'Hot-springs',
 'Lake',
 'Pub-walk',
 'Rails-trails',
 'River',
 'Views',
 'Waterfall',
 'Wildflowers',
 'Wildlife']

In [10]:
suitability_tags

['Dog-friendly',
 'Paved',
 'Kid-friendly',
 'Partially-paved',
 'Wheelchair-friendly',
 'Stroller-friendly']